In [1]:
%cd ..

/mnt/dog/data/indraroy15/2025-MatchCut


In [2]:
import sys
sys.path.append('data/MovieCuts/src')

In [3]:

config = {
    'data': {
        'videos_path': './data/MovieCuts/data/framed_clips', #NOTE:needs downloading 
        'cache_path': './.cache',
        'shots_file_train': 'data/MovieCuts/data/annotated_clips_train.csv',
        'shots_file_val': 'data/MovieCuts/data/annotated_clips_val.csv',
        'cut_type_file_name_train': 'data/MovieCuts/data/cut-type-train.json',
        'cut_type_file_name_val': 'data/MovieCuts/data/cut-type-val.json',
        'cut_type_file_name_test': 'data/MovieCuts/data/cut-type-test.json',
        'scale_h': 128,
        'scale_w': 180,
        'crop_size': 112,
        'snippet_size': 16,
        'num_classes': 10,
        'window_sampling': 'fixed',
        'distribution': 'natural',
        'data_percent': 1,
        'negative_portion': 0
    },
    'model': {
        'visual_stream': True,
        'audio_stream': True,
        'num_visual_layers': 18,
        'num_audio_layers': 18,
        'abeta': 0.6,
        'avbeta': 0.18,
        'vbeta': 0.22
    }
}

import argparse

# recursively make the dict into argparse.Namespace object 
def dict_to_namespace(d):
    if isinstance(d, dict):
        d = argparse.Namespace(**{k: dict_to_namespace(v) for k, v in d.items()})
    return d

config = dict_to_namespace(config)





In [4]:

from cut_type_dataset import CutTypeDataset

val_dataset = CutTypeDataset([config.data.shots_file_train, config.data.shots_file_val],
                config.data.cut_type_file_name_val,
                videos_path=config.data.videos_path,
                cache_path=config.data.cache_path,
                visual_stream=config.model.visual_stream,
                audio_stream=config.model.audio_stream,
                sampling=config.data.window_sampling,
                snippet_size=config.data.snippet_size,
                transform=None,
                negative_portion=config.data.negative_portion)

test_dataset = CutTypeDataset([config.data.shots_file_train, config.data.shots_file_val],
                config.data.cut_type_file_name_test,
                videos_path=config.data.videos_path,
                cache_path=config.data.cache_path,
                visual_stream=config.model.visual_stream,
                audio_stream=config.model.audio_stream,
                sampling=config.data.window_sampling,
                snippet_size=config.data.snippet_size,
                transform=None,
                negative_portion=config.data.negative_portion)

train_dataset = CutTypeDataset([config.data.shots_file_train, config.data.shots_file_val],
                config.data.cut_type_file_name_train,
                videos_path=config.data.videos_path,
                cache_path=config.data.cache_path,
                visual_stream=config.model.visual_stream,
                audio_stream=config.model.audio_stream,
                sampling=config.data.window_sampling,
                snippet_size=config.data.snippet_size,
                data_percent=config.data.data_percent,
                distribution=config.data.distribution,
                transform=None,
                negative_portion=config.data.negative_portion)

In [5]:
val_dataset.num_per_class ,test_dataset.num_per_class ,train_dataset.num_per_class 

({'cut-on-action': 2542,
  'cut-away': 1296,
  'cross-cut': 1194,
  'emphasis/deemphasis': 1538,
  'match-cut': 93,
  'smash-cut': 693,
  'reaction-in/reaction-out-cut': 9192,
  'l-cut': 1680,
  'j-cut': 2082,
  'speaker-change': 4889},
 {'cut-on-action': 5195,
  'cut-away': 2534,
  'cross-cut': 2413,
  'emphasis/deemphasis': 3079,
  'match-cut': 189,
  'smash-cut': 1459,
  'reaction-in/reaction-out-cut': 18156,
  'l-cut': 3299,
  'j-cut': 4055,
  'speaker-change': 10364},
 {'cut-on-action': 17463,
  'cut-away': 8152,
  'cross-cut': 7332,
  'emphasis/deemphasis': 10594,
  'match-cut': 591,
  'smash-cut': 4654,
  'reaction-in/reaction-out-cut': 64774,
  'l-cut': 11200,
  'j-cut': 13955,
  'speaker-change': 35449})

In [6]:
import numpy as np 
desired_cut_type = 'match-cut'
desired_cut_type_label_idx = int((np.array(val_dataset.cut_types) == desired_cut_type).nonzero()[0][0])
desired_cut_type_label_idx

4

In [ ]:
import os
target_folder =  f"{config.data.videos_path}_{desired_cut_type}"
os.makedirs(target_folder, exist_ok=True)

In [8]:
import shutil
for ref in [train_dataset, val_dataset, test_dataset]:
    
    filtered_clip_names = [clip_name for clip_name in ref.clip_names if\
                        ref.cut_type_annotations[clip_name]['labels'][desired_cut_type_label_idx] == 1]
    #Assert that the number of filtererd clips of the desired cut type is equal to the expected count
    assert len(filtered_clip_names) == ref.num_per_class[desired_cut_type]

    for clip_name in filtered_clip_names:
        source_folder = os.path.join(config.data.videos_path, clip_name)
        destination_folder = os.path.join(target_folder, clip_name)
        if os.path.exists(source_folder):
            if not os.path.exists(destination_folder):
                shutil.copytree(source_folder, destination_folder)
        else:
            print(f"Source folder {source_folder} does not exist.")

In [ ]:
#Final count check 
import glob
len(glob.glob(f"{target_folder}/*")) == (train_dataset.num_per_class[desired_cut_type]+\
                                         val_dataset.num_per_class[desired_cut_type]+\
                                         test_dataset.num_per_class[desired_cut_type])

True